In [ ]:
cfg = edict(
    {
        "data_path": "/home/ma-user/work/flowers_photos",
        "data_size": 3670,
        "image_width": 100,  # 图片宽度
        "image_height": 100,  # 图片高度
        "batch_size": 32,
        "channel": 3,  # 图片通道数
        "num_class": 5,  # 分类类别
        "weight_decay": 0.01,
        "lr": 0.0001,  # 学习率
        "dropout_ratio": 0.5,
        "epoch_size": 400,  # 训练次数
        "sigma": 0.01,
        "save_checkpoint_steps": 1,  # 多少步保存一次模型
        "keep_checkpoint_max": 1,  # 最多保存多少个模型
        "output_directory": "./",  # 保存模型路径
        "output_prefix": "checkpoint_classification",  # 保存模型文件名字
    }
)

In [ ]:
# 从目录中读取图像的源数据集。
de_dataset = ds.ImageFolderDataset(
    cfg.data_path,
    class_indexing={
        "daisy": 0,
        "dandelion": 1,
        "roses": 2,
        "sunflowers": 3,
        "tulips": 4,
    },
)
# 解码前将输入图像裁剪成任意大小和宽高比。
transform_img = CV.RandomCropDecodeResize(
    [cfg.image_width, cfg.image_height], scale=(0.08, 1.0), ratio=(0.75, 1.333)
)  # 改变尺寸
# 转换输入图像；形状（H, W, C）为形状（C, H, W）。
hwc2chw_op = CV.HWC2CHW()
# 转换为给定MindSpore数据类型的Tensor操作。
type_cast_op = C.TypeCast(mstype.float32)
# 将操作中的每个操作应用到此数据集。
de_dataset = de_dataset.map(
    input_columns="image", num_parallel_workers=8, operations=transform_img
)
de_dataset = de_dataset.map(
    input_columns="image", operations=hwc2chw_op, num_parallel_workers=8
)
de_dataset = de_dataset.map(
    input_columns="image", operations=type_cast_op, num_parallel_workers=8
)
de_dataset = de_dataset.shuffle(buffer_size=cfg.data_size)
# 划分训练集测试集
(de_train, de_test) = de_dataset.split([0.8, 0.2])
# 设置每个批处理的行数
# drop_remainder确定是否删除最后一个可能不完整的批（default=False）。
# 如果为True，并且如果可用于生成最后一个批的batch_size行小于batch_size行，则这些行将被删除，并且不会传播到子节点。
de_train = de_train.batch(cfg.batch_size, drop_remainder=True)
# 重复此数据集计数次数。
de_test = de_test.batch(cfg.batch_size, drop_remainder=True)
print(
    "训练数据集数量：", de_train.get_dataset_size() * cfg.batch_size
)  # get_dataset_size()获取批处理的大小。
print("测试数据集数量：", de_test.get_dataset_size() * cfg.batch_size)
# __next__方法处理后，获取一个batch的数据，数据格式为NCHW，第一维度为batch的数量。
data_next = de_dataset.create_dict_iterator(output_numpy=True).__next__()
print("通道数/图像长/宽：", data_next["image"].shape)
print("一张图像的标签样式：", data_next["label"])  # 一共5类，用0-4的数字表达类别。
print(data_next["image"][0, ...].shape)

plt.figure()
plt.imshow(data_next["image"][1, ...])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# 定义CNN图像识别网络
class Identification_Net(nn.Cell):
    def __init__(
        self, num_class=5, channel=3, dropout_ratio=0.5, trun_sigma=0.01
    ):  # 一共分五类，图片通道数是3
        super(Identification_Net, self).__init__()
        self.num_class = num_class
        self.channel = channel
        self.dropout_ratio = dropout_ratio
        # 设置卷积层
        self.conv1 = nn.Conv2d(
            self.channel,
            32,
            kernel_size=5,
            stride=1,
            padding=0,
            has_bias=True,
            pad_mode="same",
            weight_init=TruncatedNormal(sigma=trun_sigma),
            bias_init="zeros",
        )
        # 设置ReLU激活函数
        self.relu = nn.ReLU()
        # 设置最大池化层
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2, pad_mode="valid")
        self.conv2 = nn.Conv2d(
            32,
            64,
            kernel_size=5,
            stride=1,
            padding=0,
            has_bias=True,
            pad_mode="same",
            weight_init=TruncatedNormal(sigma=trun_sigma),
            bias_init="zeros",
        )
        self.conv3 = nn.Conv2d(
            64,
            128,
            kernel_size=3,
            stride=1,
            padding=0,
            has_bias=True,
            pad_mode="same",
            weight_init=TruncatedNormal(sigma=trun_sigma),
            bias_init="zeros",
        )
        self.conv4 = nn.Conv2d(
            128,
            128,
            kernel_size=3,
            stride=1,
            padding=0,
            has_bias=True,
            pad_mode="same",
            weight_init=TruncatedNormal(sigma=trun_sigma),
            bias_init="zeros",
        )
        self.flatten = nn.Flatten()
        self.fc1 = nn.Dense(
            6 * 6 * 128,
            1024,
            weight_init=TruncatedNormal(sigma=trun_sigma),
            bias_init=0.1,
        )
        self.dropout = nn.Dropout(self.dropout_ratio)
        self.fc2 = nn.Dense(
            1024, 512, weight_init=TruncatedNormal(sigma=trun_sigma), bias_init=0.1
        )
        self.fc3 = nn.Dense(
            512,
            self.num_class,
            weight_init=TruncatedNormal(sigma=trun_sigma),
            bias_init=0.1,
        )

    # 构建模型
    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv3(x)
        x = self.max_pool2d(x)
        x = self.conv4(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
# 实例化网络
net = Identification_Net(
    num_class=cfg.num_class, channel=cfg.channel, dropout_ratio=cfg.dropout_ratio
)
# 计算softmax交叉熵，以此作为损失函数
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
# 获取网络中需要更新的参数
fc_weight_params = list(
    filter(lambda x: "fc" in x.name and "weight" in x.name, net.trainable_params())
)
other_params = list(
    filter(
        lambda x: "fc" not in x.name or "weight" not in x.name, net.trainable_params()
    )
)
group_params = [
    {"params": fc_weight_params, "weight_decay": cfg.weight_decay},
    {"params": other_params},
    {"order_params": net.trainable_params()},
]
# 设置Adam优化器，将前一步设定的需要更新的参数传入
net_opt = nn.Adam(group_params, learning_rate=cfg.lr, weight_decay=0.0)
# net_opt = nn.Adam(params=net.trainable_params(), learning_rate=cfg.lr, weight_decay=0.1)
# 编译网络、损失函数和优化器
model = Model(net, loss_fn=net_loss, optimizer=net_opt, metrics={"acc"})

In [ ]:
# 设定callback监控指标
loss_cb = LossMonitor(per_print_times=de_train.get_dataset_size() * 10)
config_ck = CheckpointConfig(
    save_checkpoint_steps=cfg.save_checkpoint_steps,
    keep_checkpoint_max=cfg.keep_checkpoint_max,
)
ckpoint_cb = ModelCheckpoint(
    prefix=cfg.output_prefix, directory=cfg.output_directory, config=config_ck
)
print("============== Starting Training ==============")
# 开始训练，训练时将callback监控指标设定好
model.train(
    cfg.epoch_size, de_train, callbacks=[loss_cb, ckpoint_cb], dataset_sink_mode=True
)

# 使用测试集评估模型，打印总体准确率
metric = model.eval(de_test)
print(metric)

In [ ]:
# 加载模型
import os

# 获取路径
CKPT = os.path.join(
    cfg.output_directory,
    cfg.output_prefix
    + "-"
    + str(cfg.epoch_size)
    + "_"
    + str(de_train.get_dataset_size())
    + ".ckpt",
)
# 实例化网络
net = Identification_Net(
    num_class=cfg.num_class, channel=cfg.channel, dropout_ratio=cfg.dropout_ratio
)
# 将参数加载进网络
load_checkpoint(CKPT, net=net)
# 编译整个模型
model = Model(net)
# 预测
class_names = {0: "daisy", 1: "dandelion", 2: "roses", 3: "sunflowers", 4: "tulips"}
# 获取测试集的batch
test_ = de_test.create_dict_iterator().__next__()
# 转换为tensor
test = Tensor(test_["image"], mindspore.float32)
# 预测
predictions = model.predict(test)
predictions = predictions.asnumpy()
true_label = test_["label"].asnumpy()

# 显示预测结果
for i in range(9):
    p_np = predictions[i, :]
    pre_label = np.argmax(p_np)
    print(
        "第" + str(i) + "个sample预测结果：",
        class_names[pre_label],
        "真实结果：",
        class_names[true_label[i]],
    )